In [42]:
from qiskit_nature.problems.sampling.protein_folding.interactions.random_interaction import (
    RandomInteraction,
)
from qiskit_nature.problems.sampling.protein_folding.interactions.miyazawa_jernigan_interaction import (
    MiyazawaJerniganInteraction,
)
from qiskit_nature.problems.sampling.protein_folding.peptide.peptide import Peptide
from qiskit_nature.problems.sampling.protein_folding.protein_folding_problem import (
    ProteinFoldingProblem,
)

from qiskit_nature.problems.sampling.protein_folding.penalty_parameters import PenaltyParameters

from qiskit.utils import algorithm_globals, QuantumInstance

algorithm_globals.random_seed = 23

In [48]:
import pennylane as qml

def hamiltonian_from_qiskit(sum_op):

	'''
	This function takes a qiskit.PauliSumOp as input and
	gives back a Pennylane Hamiltonian as output

	sum_op  = qiskit.PauliSumOp

	'''
	op      = sum_op.primitive.to_list()
	n_terms = len(op)
	coeffs  = []
	paulis  = []

	for i in range(n_terms):
		paulis.append(qml.grouping.string_to_pauli_word(op[i][0]))
		coeffs.append(op[i][1].real)

	p_op = qml.Hamiltonian(coeffs,paulis)

	return p_op

In [230]:
#main_chain = "YPYFIP"
#side_chains = ["","I","P","F","Y",""] 
main_chain = "YFIP"
side_chains = ["","F","Y",""] 

In [231]:
random_interaction = RandomInteraction()
mj_interaction = MiyazawaJerniganInteraction()

In [232]:
penalty_back = 10
penalty_chiral = 10
penalty_1 = 10

penalty_terms = PenaltyParameters(penalty_chiral, penalty_back, penalty_1)

In [233]:
peptide = Peptide(main_chain, side_chains)

In [234]:
protein_folding_problem = ProteinFoldingProblem(peptide, mj_interaction, penalty_terms)
qubit_op = protein_folding_problem.qubit_op()



In [236]:
cost_h=hamiltonian_from_qiskit(qubit_op)
mixer_h=qml.qaoa.x_mixer(wires=range(0,6,1))

In [237]:
import pennylane as qml
from pennylane import numpy as np

#cost_h, mixer_h = qml.qaoa.max_clique(g, constrained=False)
# constrained=True results in greater circuit depth but potentially better solutions

print("Cost Hamiltonian:\n", cost_h)
print("Mixer Hamiltonian:\n", mixer_h)

Cost Hamiltonian:
   (0.625) [Z5]
+ (0.625) [Z4]
+ (0.625) [Z0]
+ (0.625) [Z1]
+ (2.5) [Z2]
+ (2.5) [Z3]
+ (15.625) [I0]
+ (-2.5) [Z2 Z3]
+ (-1.875) [Z1 Z4]
+ (0.625) [Z4 Z5]
+ (0.625) [Z0 Z4]
+ (0.625) [Z0 Z5]
+ (0.625) [Z1 Z5]
+ (0.625) [Z0 Z1]
+ (-1.875) [Z0 Z4 Z5]
+ (-1.875) [Z0 Z1 Z5]
+ (0.625) [Z1 Z4 Z5]
+ (0.625) [Z0 Z1 Z4]
+ (0.625) [Z0 Z1 Z4 Z5]
Mixer Hamiltonian:
   (1) [X0]
+ (1) [X1]
+ (1) [X2]
+ (1) [X3]
+ (1) [X4]
+ (1) [X5]


In [239]:
n_nodes = 6

In [240]:
def qaoa_layer(gamma, alpha):
    qml.qaoa.cost_layer(gamma, cost_h)
    qml.qaoa.mixer_layer(alpha, mixer_h)

In [241]:
n_layers = 1
wires = range(n_nodes)

def circuit(params, **kwargs):
    for w in wires:  # Prepare an equal superposition over all qubits
        qml.Hadamard(wires=w)
        #print(w)
        
    qml.layer(qaoa_layer, n_layers, params[0], params[1])


In [242]:
np.random.seed(1967)
params = np.random.uniform(size=[2, n_layers])
params

tensor([[0.72511958],
        [0.57312068]], requires_grad=True)

In [243]:
dev = qml.device("qulacs.simulator", wires=wires)

In [244]:
@qml.qnode(dev)
def cost_function(params, **kwargs):
    circuit(params=params)
    return qml.expval(cost_h)

def cvar_cost(params,shots,alpha):
    h=[]
    for i in range(shots):
        h.append(cost_function(params=params))
    h.sort()
    return np.mean(h[:int(shots*alpha)])
    
    
        

In [245]:
#import scipy
#optimizer = qml.RotosolveOptimizer()
optimizer = qml.GradientDescentOptimizer()


In [246]:
print("Initial cost:", cvar_cost(params,10,0.1))


Initial cost: 17.407052624126937


In [229]:
print("Initial cost:", cost_function(params))

for i in range(10):
    params = optimizer.step(cost_function, params)
    cost_eval = cost_function(params)
    print(f"Completed iteration {i + 1}, cost function:", cost_eval)

Initial cost: 2225.2746125879944


KeyboardInterrupt: 